In [1]:
from abc import abstractmethod
from dataclasses import InitVar, dataclass, field
from pathlib import Path
from typing import Iterable, NamedTuple, Sequence, TypeAlias

from lightning import pytorch as pl
import numpy as np
import pandas as pd
from rdkit.Chem import Mol
import torch
from torch import Tensor, nn
from torch.utils.data import DataLoader, Dataset

from chemprop.data.collate import BatchMolGraph, collate_batch
from chemprop.data.datapoints import MoleculeDatapoint
from chemprop.data.datasets import Datum, MoleculeDataset, MulticomponentDataset, ReactionDataset
from chemprop.data.molgraph import MolGraph
from chemprop.data.splitting import make_split_indices, split_data_by_indices
from chemprop.featurizers import Featurizer, SimpleMoleculeMolGraphFeaturizer
from chemprop.models import MulticomponentMPNN, multi
from chemprop.nn.agg import Aggregation, MeanAggregation
from chemprop.nn.hparams import HasHParams
from chemprop.nn.message_passing import BondMessagePassing, MulticomponentMessagePassing, MessagePassing
from chemprop.nn.metrics import ChempropMetric
from chemprop.nn.predictors import Predictor, RegressionFFN
from chemprop.nn.transforms import ScaleTransform, UnscaleTransform
from chemprop.nn.utils import Activation, get_activation_function
from lightning.pytorch.core.mixins import HyperparametersMixin
from chemprop.conf import DEFAULT_ATOM_FDIM, DEFAULT_BOND_FDIM, DEFAULT_HIDDEN_DIM


We first extend the `MolGraph` class to include a global attribute `w_fp` which is the weight of the learned fingerprint of the molecule when averaging the fingerprints of components in the mixture.

In [2]:
# See also chemprop.data.molgraph.MolGraph
class ComponentMolGraph(NamedTuple):
    V: np.ndarray
    E: np.ndarray
    edge_index: np.ndarray
    rev_edge_index: np.ndarray
    w_fp: float = 1.0
    """the weight of the component's fingerprint when combining components in the mixture"""


# See also chemprop.data.datasets.Datum
class ComponentDatum(NamedTuple):
    mg: ComponentMolGraph
    V_d: np.ndarray | None
    x_d: np.ndarray | None
    y: np.ndarray | None
    weight: float
    lt_mask: np.ndarray | None
    gt_mask: np.ndarray | None

The batched versions of `MolGraph` and `Datum` are created during collating datapoints. These are also extended, as well as the entire batch representing all components in the mixture.

In [3]:
# See also chemprop.data.collate.BatchMolGraph
@dataclass(repr=False, eq=False, slots=True)
class BatchComponentMolGraph(BatchMolGraph):
    mgs: InitVar[Sequence[ComponentMolGraph]]
    w_fps: Tensor = field(init=False)

    def __post_init__(self, mgs):
        super(BatchComponentMolGraph, self).__post_init__(mgs)
        self.w_fps = torch.from_numpy(np.array([mg.w_fp for mg in mgs])).float()

    def to(self, device: str | torch.device):
        super(BatchComponentMolGraph, self).to(device)
        self.w_fps = self.w_fps.to(device)


# See also chemprop.data.collate.TrainingBatch
class BatchComponentDatum(NamedTuple):
    bmg: BatchComponentMolGraph
    V_d: Tensor | None
    X_d: Tensor | None
    Y: Tensor | None
    w: Tensor
    lt_mask: Tensor | None
    gt_mask: Tensor | None


# See also chemprop.data.collate.MulticomponentTrainingBatch
class MixtureBatch(NamedTuple):
    bmgs: list[BatchMolGraph | BatchComponentMolGraph]
    V_ds: list[Tensor | None]
    X_d: Tensor | None
    Y: Tensor | None
    w: Tensor
    lt_mask: Tensor | None
    gt_mask: Tensor | None

In [4]:
# See also chemprop.data.collate.collate_batch
def collate_component(batch: Iterable[Datum]) -> BatchComponentDatum:
    mgs, V_ds, x_ds, ys, weights, lt_masks, gt_masks = zip(*batch)

    return BatchComponentDatum(
        BatchComponentMolGraph(mgs),
        None if V_ds[0] is None else torch.from_numpy(np.concatenate(V_ds)).float(),
        None if x_ds[0] is None else torch.from_numpy(np.array(x_ds)).float(),
        None if ys[0] is None else torch.from_numpy(np.array(ys)).float(),
        torch.tensor(weights, dtype=torch.float).unsqueeze(1),
        None if lt_masks[0] is None else torch.from_numpy(np.array(lt_masks)),
        None if gt_masks[0] is None else torch.from_numpy(np.array(gt_masks)),
    )


# See also chemprop.data.collate.collate_multicomponent
def collate_mixture(batches: Iterable[Iterable[ComponentDatum | Datum]]) -> MixtureBatch:
    tbs = [
        collate_batch(batch) if isinstance(batch[0], Datum) else collate_component(batch)
        for batch in zip(*batches)
    ]

    return MixtureBatch(
        [tb.bmg for tb in tbs],
        [tb.V_d for tb in tbs],
        tbs[0].X_d,
        tbs[0].Y,
        tbs[0].w,
        tbs[0].lt_mask,
        tbs[0].gt_mask,
    )

In [5]:
@dataclass
class ComponentDatapoint(MoleculeDatapoint):
    w_fp: np.ndarray | None = None
    """the weight of the molecule's learned fingerprint when averaging in the mixture"""


@dataclass
class ComponentDataset(MoleculeDataset, Dataset[ComponentMolGraph]):
    data: list[ComponentDatapoint]

    @property
    def w_fps(self) -> np.ndarray:
        return np.array([d.w_fp for d in self.data])

    def __getitem__(self, idx: int) -> ComponentDatum:
        d = self.data[idx]
        mg = self.mg_cache[idx]
        mg = ComponentMolGraph(w_fp=d.w_fp, *mg)

        return ComponentDatum(
            mg, self.V_ds[idx], self.X_d[idx], self.Y[idx], d.weight, d.lt_mask, d.gt_mask
        )


@dataclass(repr=False, eq=False)
class MixtureDataset(MulticomponentDataset):
    datasets: list[MoleculeDataset | ReactionDataset | ComponentDataset]

    def __getitem__(self, idx: int) -> list[ComponentDatum | Datum]:
        return [dset[idx] for dset in self.datasets]

In [6]:
class MixtureMessagePassing(nn.Module, HyperparametersMixin):
    r"""A :class:`MixtureMessagePassing` encodes a batch of mixtures by passing messages along
    molecules constructing a fully connected graph to model intermolecular interactions.

    It implements the following operation:

    .. math::

        h_v^{(0)} &= \tau \left( \mathbf{W}_i(x_v) \right) \\
        m_v^{(t)} &= \sum_{u \in \mathcal{w \in V \setminu v} h_w^{(t-1)} \\
        h_v^{(T)} &= \tau\left(h_v^{(0)} + \mathbf{W}_h m_v^{(t-1)}\right) \\

    where :math:`\tau` is the activation function; :math:`\mathbf{W}_i`, :math:`\mathbf{W}_h` are learned weight matrices; :math:`e_{vw}` is the feature vector of the
    bond between molecules :math:`v` and :math:`w`; :math:`x_v` is the feature vector of molecule :math:`v`;
    :math:`h_v^{(t)}` is the hidden representation of atom :math:`v` at iteration :math:`t`;
    :math:`m_v^{(t)}` is the message received by atom :math:`v` at iteration :math:`t`; and
    :math:`t \in \{1, \dots, T\}` is the number of message passing iterations.
    """

    def __init__(
        self,
        d_v: int = DEFAULT_HIDDEN_DIM,
        d_e: int | None = None,
        d_h: int = DEFAULT_HIDDEN_DIM,
        d_vd: int | None = None,
        bias: bool = False,
        depth: int = 1,
        activation: str | Activation = Activation.RELU,
    ):        
        super().__init__()
        self.save_hyperparameters()
        self.hparams["cls"] = self.__class__

        self.depth = depth
        self.tau = get_activation_function(activation)
        
        self.W_i = nn.Linear(d_v, d_h, bias)
        self.W_h = nn.Linear(d_h, d_h, bias) # TODO consider E
        self.W_o = None
        self.W_d = None

    def initialize(self, V: Tensor) -> Tensor:
        return self.W_i(V)

    def message(self, H: Tensor):
        # assume fully connected graph, TODO
        H = torch.transpose(H, 0, 1) # b x n x d
        M_t = H.unsqueeze(2).expand(-1, -1, H.size(1), -1)
        M_t = self.W_h(M_t)
        mask = ~torch.eye(H.size(1), dtype=bool, device=H.device).unsqueeze(0) # exclude self-loops
        M_t = (M_t * mask.unsqueeze(-1)).sum(dim=1)
        M_t = torch.transpose(M_t, 0, 1)
        return M_t

    def update(self, M_t: Tensor, H_0: Tensor):
        H_t = self.tau(H_0 + M_t)
        return H_t

    def finalize(self, H_t: Tensor):
        return [h for h in H_t]

    def forward(self, V: list[Tensor]):
        H_0 = self.initialize(torch.stack(V))
        H = self.tau(H_0)
        for _ in range(self.depth):
            M = self.message(H)
            H = self.update(M, H_0)

        return self.finalize(H)


In [26]:
class MixtureAggregation(nn.Module, HasHParams):
    output_dim: int

    def __init__(
        self, 
        graph_agg: Aggregation, 
        groups: Sequence[Sequence[int]], 
        fp_dims: Sequence[int], 
        mixmp: MixtureMessagePassing | None, 
        *args, 
        **kwargs
    ):
        super().__init__()
        self.hparams = {
            "cls": self.__class__,
            "groups": groups,
            "fp_dims": fp_dims,
        }
        self.graph_agg = graph_agg
        self.groups = groups
        self.fp_dims = fp_dims
        self.mixmp = mixmp

    @abstractmethod
    def forward(
        self, Hs: list[Tensor], bmgs: list[BatchComponentMolGraph | BatchMolGraph]
    ) -> Tensor:
        """Aggregate component representations into a mixture representation"""

class WeightedSumAggregation(MixtureAggregation):
    @property
    def output_dim(self) -> int:
        return sum(self.fp_dims[group[0]] for group in self.groups)

    def forward(
        self, H_vs: list[Tensor], bmgs: list[BatchComponentMolGraph | BatchMolGraph]
    ) -> Tensor:
        Hs = [self.graph_agg(H_v, bmg.batch) for H_v, bmg in zip(H_vs, bmgs)]
        if not (self.mixmp is None):
            Hs = self.mixmp(Hs)
        combined_Hs = []
        for group in self.groups:
            if len(group) == 1:
                combined_Hs.append(Hs[group[0]])
                continue

            group_Hs = torch.stack([Hs[idx] for idx in group])  # n x b x d
            group_w_fps = torch.stack([bmgs[idx].w_fps for idx in group])  # n x b
            # n: num. components in group, b: num. comp. in batch, d: output dim of message passing
            combined_H = torch.einsum("nb,nbd->bd", group_w_fps, group_Hs)
            combined_Hs.append(combined_H)
        return torch.cat(combined_Hs, 1)

class ConcatAggregation(MixtureAggregation):
    @property
    def components_in_mixture(self) -> set[int]:
        return {idx for group in self.groups if len(group) > 1 for idx in group}

    @property
    def output_dim(self) -> int:
        return sum(self.fp_dims) + len(self.components_in_mixture)

    def forward(
        self, H_vs: list[Tensor], bmgs: list[BatchComponentMolGraph | BatchMolGraph]
    ) -> Tensor:
        Hs = [self.graph_agg(H_v, bmg.batch) for H_v, bmg in zip(H_vs, bmgs)]
        if not (self.mixmp is None):
            Hs = self.mixmp(Hs)
        w_fps = torch.stack([bmgs[idx].w_fps for idx in self.components_in_mixture], dim=1)
        return torch.cat(Hs + [w_fps], 1)

class DeepsetsAggregation(MixtureAggregation):
    r"""Deep sets aggregation of the graph-level representation:

    .. math::
        \mathbf h = \mathrm{MLP_{g}}(\sum_{c \in C} \mathrm{MLP_{l}}(\mathbf h_c))
    """
    def __init__(
        self, graph_agg: Aggregation, groups: Sequence[Sequence[int]], fp_dims: Sequence[int], 
        mixmp: MixtureMessagePassing | None, *args, **kwargs
    ):
        super().__init__(graph_agg, groups, fp_dims, mixmp, *args, **kwargs)
        
        self.MLPs_local = nn.ModuleList([])
        self.MLPs_global = nn.ModuleList([])
        for group in groups:
            # TODO: allow to set hparams for MLP by kwargs (e.g., hidden_dim, n_layers)
            hidden_dim = self.fp_dims[group[0]]
            self.MLPs_local.append(
                nn.Sequential(
                nn.Linear(self.fp_dims[group[0]], hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, self.fp_dims[group[0]]),
                )
            )
            self.MLPs_global.append(
                nn.Sequential(
                nn.Linear(self.fp_dims[group[0]], hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, self.fp_dims[group[0]]),
                )
            )

    @property
    def output_dim(self) -> int:
        return sum(self.fp_dims[group[0]] for group in self.groups)

    def forward(
        self, H_vs: list[Tensor], bmgs: list[BatchComponentMolGraph | BatchMolGraph]
    ) -> Tensor:
        Hs = [self.graph_agg(H_v, bmg.batch) for H_v, bmg in zip(H_vs, bmgs)]
        if not (self.mixmp is None):
            Hs = self.mixmp(Hs)
        combined_Hs = []
        for g_idx, group in enumerate(self.groups):
            if len(group) == 1:
                combined_Hs.append(self.MLPs_global[g_idx](Hs[group[0]]))
                continue
            group_w_Hs = torch.stack([self.MLPs_local[g_idx](bmgs[idx].w_fps.unsqueeze(1) * Hs[idx]) for idx in group])  # n x b x d
            combined_H = torch.sum(group_w_Hs, dim=0)
            combined_Hs.append(self.MLPs_global[g_idx](combined_H))
        return torch.cat(combined_Hs, 1)

class AttentiveAggregation(MixtureAggregation):
    r"""Attentive aggregation of the graph-level representation:

    .. math::
        \mathbf h = \sum_{c \in C} \alpha_c \mathbf h_c

        \alpha_c = \mathrm{softmax}(\mathbf h_c)
    """
    def __init__(
        self, graph_agg: Aggregation, groups: Sequence[Sequence[int]], fp_dims: Sequence[int], 
        mixmp: MixtureMessagePassing | None, *args, **kwargs
    ):
        super().__init__(graph_agg, groups, fp_dims, mixmp, *args, **kwargs)
        
        self.Ws_a = nn.ModuleList([
            nn.Linear(self.fp_dims[group[0]], 1) for group in groups
            ])

    @property
    def output_dim(self) -> int:
        return sum(self.fp_dims[group[0]] for group in self.groups)

    def forward(
        self, H_vs: list[Tensor], bmgs: list[BatchComponentMolGraph | BatchMolGraph]
    ) -> Tensor:
        Hs = [self.graph_agg(H_v, bmg.batch) for H_v, bmg in zip(H_vs, bmgs)]
        if not (self.mixmp is None):
            Hs = self.mixmp(Hs)
        combined_Hs = []
        for g_idx, group in enumerate(self.groups):
            if len(group) == 1:
                combined_Hs.append((Hs[group[0]]))
                continue
            w_Hs = torch.stack([bmgs[idx].w_fps.unsqueeze(1) * Hs[idx] for idx in group])  # n x b x d
            attention_logits = self.Ws_a[g_idx](w_Hs).exp()
            Z = torch.sum(attention_logits, dim=0, keepdim=False)
            alphas = attention_logits / Z
            combined_H = torch.sum(alphas * w_Hs, dim=0)
            combined_Hs.append(combined_H)
        return torch.cat(combined_Hs, 1)

class Set2SetAggregation(MixtureAggregation):
    r"""Set2Set aggregation of the graph-level representation:

    .. math::
        \mathbf{q}_t &= \mathrm{LSTM}(\mathbf{q}^{*}_{t-1})

        \alpha_{c,t} &= \mathrm{softmax}(\mathbf{h}_c \cdot \mathbf{q}_t)

        \mathbf{r}_t &= \sum_{c=1}^C \alpha_{c,t} \mathbf{h}_c

        \mathbf{q}^{*}_t &= \mathbf{q}_t \, \Vert \, \mathbf{r}_t,

    where :math:`\mathbf{q}^{*}_T` defines the output of the layer with twice
    the dimensionality as the input.
    
    Note: This implementation follows PyTorch Geometric (cf. https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/aggr/set2set.html#Set2Set) and is based on `"Order Matters: Sequence to sequence for
    Sets" <https://arxiv.org/abs/1511.06391>`_ paper.
    """
    def __init__(
        self, graph_agg: Aggregation, groups: Sequence[Sequence[int]], fp_dims: Sequence[int], 
        mixmp: MixtureMessagePassing | None, *args, **kwargs
    ):
        super().__init__(graph_agg, groups, fp_dims, mixmp, *args, **kwargs)
        
        # TODO: allow to set hparams for Set2Set by kwargs (e.g., processing steps)
        self.processing_steps = 3
        self.lstms = nn.ModuleList([])
        for group in groups:
            in_channels = self.fp_dims[group[0]]
            out_channels = self.fp_dims[group[0]] * 2
            self.lstms.append(
                torch.nn.LSTM(out_channels, in_channels, **kwargs)
                )

    @property
    def output_dim(self) -> int:
        return sum(self.fp_dims[group[0]] * 2 if len(group) > 1 else self.fp_dims[group[0]] for group in self.groups)

    def forward(
        self, H_vs: list[Tensor], bmgs: list[BatchComponentMolGraph | BatchMolGraph]
    ) -> Tensor:
        Hs = [self.graph_agg(H_v, bmg.batch) for H_v, bmg in zip(H_vs, bmgs)]
        if not (self.mixmp is None):
            Hs = self.mixmp(Hs)
        combined_Hs = []
        for g_idx, group in enumerate(self.groups):
            if len(group) == 1:
                combined_Hs.append((Hs[group[0]]))
                continue
            
            w_Hs = torch.stack([bmgs[idx].w_fps.unsqueeze(1) * Hs[idx] for idx in group]) 
            w_Hs = torch.transpose(w_Hs, 0, 1) # b x n x d
            b_din = w_Hs.size(0)
            d_dim = w_Hs.size(-1)

            h = (w_Hs.new_zeros((self.lstms[g_idx].num_layers, b_din, d_dim)),
                w_Hs.new_zeros((self.lstms[g_idx].num_layers, b_din, d_dim)))
            q_star = w_Hs.new_zeros(b_din, d_dim * 2)

            for _ in range(self.processing_steps):
                q, h = self.lstms[g_idx](q_star.unsqueeze(0), h)

                q = q.squeeze(0) # b x d
                e = torch.sum(w_Hs * q.unsqueeze(1), dim=2) # b x n
                a = torch.softmax(e, dim=1) # b x n
                r = torch.sum(w_Hs * a.unsqueeze(2), dim=1) # b x d
                q_star = torch.cat([q, r], dim=1) # b x 2*d

            combined_Hs.append(q_star)
        return torch.cat(combined_Hs, 1)

In [48]:
class MixtureMPNN(MulticomponentMPNN):
    def __init__(
        self,
        message_passing: MulticomponentMessagePassing,
        agg: Aggregation,
        predictor: Predictor,
        mix_mpn: MixtureMessagePassing | None = None,
        batch_norm: bool = False,
        metrics: Iterable[ChempropMetric] | None = None,
        warmup_epochs: int = 2,
        init_lr: float = 1e-4,
        max_lr: float = 1e-3,
        final_lr: float = 1e-4,
        X_d_transform: ScaleTransform | None = None,
    ):
        super().__init__(
            message_passing,
            agg,
            predictor,
            batch_norm,
            metrics,
            warmup_epochs,
            init_lr,
            max_lr,
            final_lr,
            X_d_transform,
        )
        self.agg: MixtureAggregation

    def fingerprint(
        self,
        bmgs: Iterable[BatchComponentMolGraph | BatchMolGraph],
        V_ds: Iterable[Tensor],
        X_d: Tensor | None = None,
    ) -> Tensor:
        H_vs: list[Tensor] = self.message_passing(bmgs, V_ds)
        H = self.agg(H_vs, bmgs)
        H = self.bn(H)
        return H if X_d is None else torch.cat((H, X_d), 1)

    @classmethod
    def _load(cls, path, map_location, **submodules):
        d = torch.load(path, map_location, weights_only=False)

        try:
            hparams = d["hyper_parameters"]
            state_dict = d["state_dict"]
        except KeyError:
            raise KeyError(f"Could not find hyper parameters and/or state dict in {path}.")

        hparams["message_passing"]["blocks"] = [
            block_hparams.pop("cls")(**block_hparams)
            for block_hparams in hparams["message_passing"]["blocks"]
        ]
        graph_agg_hparams = hparams["agg"]["graph_agg"]
        hparams["agg"]["graph_agg"] = graph_agg_hparams.pop("cls")(**graph_agg_hparams)
        mixmp_hparams = hparams["agg"]["mixmp"]
        hparams["agg"]["mixmp"] = mixmp_hparams.pop("cls")(**mixmp_hparams)
        submodules |= {
            key: hparams[key].pop("cls")(**hparams[key])
            for key in ("message_passing", "agg", "predictor")
            if key not in submodules
        }

        if not hasattr(submodules["predictor"].criterion, "_defaults"):
            submodules["predictor"].criterion = submodules["predictor"].criterion.__class__(
                task_weights=submodules["predictor"].criterion.task_weights
            )

        return submodules, state_dict, hparams

In [49]:
chemprop_dir = Path.cwd().parent
input_path = (
    chemprop_dir / "tests" / "data" / "regression" / "mol+mol" / "mol+mol.csv"
)  # path to your data .csv file containing SMILES strings and target values
smiles_columns = ["smiles", "solvent"]  # name of the column containing SMILES strings
target_columns = ["peakwavs_max"]  # list of names of the columns containing targets
df_input = pd.read_csv(input_path)
smiss = df_input.loc[:, smiles_columns].values
ys = df_input.loc[:, target_columns].values

In [50]:
all_data = [[MoleculeDatapoint.from_smi(smis[0], y) for smis, y in zip(smiss, ys)]]
all_data += [[ComponentDatapoint.from_smi(smis[0], w_fp=0.1) for smis in smiss]]
all_data += [[ComponentDatapoint.from_smi(smis[1], w_fp=0.9) for smis in smiss]]

In [51]:
component_to_split_by = 0  # index of the component to use for structure based splits
mols = [d.mol for d in all_data[component_to_split_by]]
train_indices, val_indices, test_indices = make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)
train_data = train_data[0]
val_data = val_data[0]
test_data = test_data[0]

The return type of make_split_indices has changed in v2.1 - see help(make_split_indices)


In [52]:
train_datasets = [
    MoleculeDataset(train_data[0]),
    ComponentDataset(train_data[1]),
    ComponentDataset(train_data[2]),
]
val_datasets = [
    MoleculeDataset(val_data[0]),
    ComponentDataset(val_data[1]),
    ComponentDataset(val_data[2]),
]
test_datasets = [
    MoleculeDataset(test_data[0]),
    ComponentDataset(test_data[1]),
    ComponentDataset(test_data[2]),
]

In [53]:
train_mcdset = MixtureDataset(train_datasets)
scaler = train_mcdset.normalize_targets()
val_mcdset = MixtureDataset(val_datasets)
val_mcdset.normalize_targets(scaler)
test_mcdset = MixtureDataset(test_datasets)

In [54]:
train_loader = DataLoader(train_mcdset, batch_size=10, shuffle=True, collate_fn=collate_mixture)
val_loader = DataLoader(val_mcdset, batch_size=10, shuffle=False, collate_fn=collate_mixture)
test_loader = DataLoader(test_mcdset, batch_size=10, shuffle=False, collate_fn=collate_mixture)

In [66]:
mcmp = MulticomponentMessagePassing(blocks=[BondMessagePassing()], n_components=3, shared=True)
mixmp = MixtureMessagePassing(d_v=mcmp.blocks[0].output_dim, d_h=mcmp.blocks[0].output_dim, depth=1)

graph_agg = MeanAggregation()
name_agg = "weightedsum"

match name_agg:
    case "weightedsum":
        mixagg = WeightedSumAggregation(
            graph_agg=graph_agg, groups=[[0], [1, 2]], fp_dims=[mcmp.blocks[0].output_dim] * 3, mixmp=mixmp,
        )
    case "cat":
        mixagg = ConcatAggregation(
            graph_agg=graph_agg, groups=[[0], [1, 2]], fp_dims=[mcmp.blocks[0].output_dim] * 3, mixmp=mixmp,
        )
    case "deepsets":
        mixagg = DeepsetsAggregation(
            graph_agg=graph_agg, groups=[[0], [1, 2]], fp_dims=[mcmp.blocks[0].output_dim] * 3, mixmp=mixmp,
        )
    case "attentive":
        mixagg = AttentiveAggregation(
            graph_agg=graph_agg, groups=[[0], [1, 2]], fp_dims=[mcmp.blocks[0].output_dim] * 3, mixmp=mixmp,
        )
    case "set2set":
        mixagg = Set2SetAggregation(
            graph_agg=graph_agg, groups=[[0], [1, 2]], fp_dims=[mcmp.blocks[0].output_dim] * 3, mixmp=mixmp,
        )
    case _:
        raise ValueError(f"MixtureAggregation {name_agg} not implemented yet.")

output_transform = UnscaleTransform.from_standard_scaler(scaler)
ffn = RegressionFFN(input_dim=mixagg.output_dim, output_transform=output_transform)

In [67]:
mcmpnn = MixtureMPNN(mcmp, mixagg, ffn)
mcmpnn

MixtureMPNN(
  (message_passing): MulticomponentMessagePassing(
    (blocks): ModuleList(
      (0-2): 3 x BondMessagePassing(
        (W_i): Linear(in_features=86, out_features=300, bias=False)
        (W_h): Linear(in_features=300, out_features=300, bias=False)
        (W_o): Linear(in_features=372, out_features=300, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (tau): ReLU()
        (V_d_transform): Identity()
        (graph_transform): Identity()
      )
    )
  )
  (agg): WeightedSumAggregation(
    (graph_agg): MeanAggregation()
  )
  (bn): Identity()
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=600, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform)

In [68]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=50,
)

/home/rwth1232/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/rwth1232/anaconda3/envs/chemprop/lib/python3.1 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [69]:
trainer.fit(mcmpnn, train_loader, val_loader)

/home/rwth1232/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /rwthfs/rz/cluster/home/rwth1232/chemprop_pool/chemprop/examples/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Loading `train_dataloader` to estimate number of stepping batches.
/home/rwth1232/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.

  | Name            | Type                         | Params | Mode 
-------------------------------------------------------------------------
0 | message_passing | MulticomponentMessagePassing | 227 K  | train
1 | agg             | WeightedSumAggregation       | 0      | train
2 | bn              | Identity  

/home/rwth1232/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Epoch 49: 100%|██████████| 8/8 [00:00<00:00, 61.26it/s, train_loss_step=0.184, val_loss=0.725, train_loss_epoch=0.0888] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 8/8 [00:00<00:00, 49.88it/s, train_loss_step=0.184, val_loss=0.725, train_loss_epoch=0.0888]


: 

In [ ]:
results = trainer.test(mcmpnn, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/rwth1232/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 88.86it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mse          │     6055.58837890625      │
└───────────────────────────┴───────────────────────────┘

In [ ]:
results = trainer.predict(mcmpnn, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/rwth1232/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 168.66it/s]


In [ ]:
results

[tensor([[399.0343],
         [708.6262],
         [366.0291],
         [397.4417],
         [344.3376],
         [373.5764],
         [358.9327],
         [567.5590],
         [368.2164],
         [509.5162]])]

In [ ]:
test_loader.dataset.datasets[0].Y

array([[384. ],
       [553. ],
       [394. ],
       [428.2],
       [386. ],
       [369. ],
       [520. ],
       [515. ],
       [313. ],
       [480. ]])

: 